In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import mplfinance as mpf
import os
from pandas import to_datetime

In [2]:
data_test = pd.read_excel(r'C:\Users\USER\Paper\Test_2023.xlsx')

In [3]:
df = data_test.rename(columns={'年月日':'Date',
                              '開盤價(元)':'Open',
                              '最高價(元)':'High',
                              '最低價(元)':'Low',
                              '收盤價(元)':'Close',})

In [4]:
df_drop = df.drop_duplicates('代號',keep='first').reset_index(drop=True)

In [5]:
len(df_drop['代號'])

1815

In [6]:
df['Date'] = pd.to_datetime(df['Date'])

In [7]:
df

,代號,名稱,Date,Open,High,Low,Close,成交值(千元),全額交割(Y)
0,Y9999,加權指數,2022-07-15,14483.96,14579.73,14378.33,14550.62,231695665,NaN
1,Y9999,加權指數,2022-07-18,14589.10,14719.64,14589.10,14719.64,229912658,NaN
2,Y9999,加權指數,2022-07-19,14638.45,14720.13,14598.99,14694.08,199312047,NaN
3,Y9999,加權指數,2022-07-20,14810.40,14906.95,14694.20,14733.22,259238995,NaN
4,Y9999,加權指數,2022-07-21,14744.17,14937.70,14716.02,14937.70,238836209,NaN
...,...,...,...,...,...,...,...,...,...
639745,9962,有益,2023-12-25,18.60,19.10,18.45,18.45,19349,NaN
639746,9962,有益,2023-12-26,18.55,18.65,18.30,18.45,7201,NaN
639747,9962,有益,2023-12-27,18.50,18.85,18.20,18.35,11151,NaN
639748,9962,有益,2023-12-28,18.55,18.55,18.25,18.25,4810,NaN


In [22]:
figure_paths = []
close_price = []
index_list = []
test_scale_min = 1800
test_scale_max = 1815
for code in df_drop['代號'].values[test_scale_min:test_scale_max]:
    draw_df = df[df['代號'] == code].reset_index(drop=True) 

    for i in range(len(draw_df)-120):
        
        stock = draw_df[i:i+120]
        stock = stock.iloc[:,2:].set_index('Date')

        if not os.path.exists(code):
            os.makedirs(code)
        mc = mpf.make_marketcolors(up='r', down='g', inherit=True)
        s  = mpf.make_mpf_style(base_mpf_style='yahoo', marketcolors=mc)
        mpf.plot(stock,
                type='candle',
                volume=False,
                figratio=(15,8),
                figscale=1,
                style=s,
                axisoff = True,
                savefig=os.path.join(code, code + '_' + str(i) + '.png'))
        figure_paths.append(os.path.join(code, code + '_' + str(i) + '.png'))
        close_price.append(stock['Close'].iloc[-1])
        index_list.append(stock.index[-1])
        
figure_data = pd.DataFrame({'File Paths': figure_paths,'Close':close_price,'Date': index_list})
figure_data['code'] = figure_data.iloc[:, 0].str.split("\\").str[0] 
figure_data.to_csv('Test_data' + '_' + str(test_scale_min) + '_' + str(test_scale_max) +'.csv')